In [1]:
import sqlite3

import numpy as np
import pandas as pd
from pathlib import Path

In [2]:
__file__ = %pwd
DATA = Path(__file__).parent / "data"
MATH = DATA / "math"
DATABASE = DATA / "databases" / "exploration.db"
DB_INSERT = False

In [3]:
math24 = pd.read_excel(MATH / "mathdownload2024.xlsx")
math23 = pd.read_excel(MATH / "mathdownload2023.xlsx")
math22 = pd.read_excel(MATH / "mathdownload2022.xlsx")
math19 = pd.read_excel(MATH / "mathdownload2019.xlsx")
math18 = pd.read_excel(MATH / "mathdownload2018.xlsx")

math24 = math24[(math24["charter_flag"] != "Y") & (math24["coe_flag"] != "Y") &
                (math24["dass_flag"] != "Y") & (math24["cds"] != 0) & (math24["cds"] != 1)]
math23 = math23[(math23["charter_flag"] != "Y") & (math23["coe_flag"] != "Y") &
                (math23["dass_flag"] != "Y") & (math23["cds"] != 0) & (math23["cds"] != 1)]
math22 = math22[(math22["charter_flag"] != "Y") & (math22["coe_flag"] != "Y") &
                (math22["dass_flag"] != "Y") & (math22["cds"] != 0) & (math22["cds"] != 1)]
math19 = math19[(math19["charter_flag"] != "Y") & (math19["coe_flag"] != "Y") &
                (math19["dass_flag"] != "Y") & (math19["cds"] != 0) & (math19["cds"] != 1)]
math18 = math18[(math18["charter_flag"] != "Y") & (math18["coe_flag"] != "Y") &
                (math18["dass_flag"] != "Y") & (math18["cds"] != 0) & (math18["cds"] != 1)]

math17 = math18[["cds", "priordenom", "priorstatus", "studentgroup"]].copy()

In [4]:
math24["Year"] = "2023-24"
math23["Year"] = "2022-23"
math22["Year"] = "2021-22"
math19["Year"] = "2018-19"
math18["Year"] = "2017-18"
math17["Year"] = "2016-17"

In [5]:
math24 = math24[["cds", "currdenom", "currstatus", "studentgroup", "Year"]]
math23 = math23[["cds", "currdenom", "currstatus", "studentgroup", "Year"]]
math22 = math22[["cds", "currdenom", "currstatus", "studentgroup", "Year"]]
math19 = math19[["cds", "currdenom", "currstatus", "studentgroup", "Year"]]
math18 = math18[["cds", "currdenom", "currstatus", "studentgroup", "Year"]]

math24.columns = math23.columns = math22.columns = ["cdsCode", "currdenom", "currstatus", "studentgroup", "Year"]
math19.columns = math18.columns = math17.columns = ["cdsCode", "currdenom", "currstatus", "studentgroup", "Year"]

In [6]:
def get_countyCode(cds_code: int) -> int:
    county_digits = 1e12
    return int(cds_code / county_digits)

def get_districtCode(cds_code: int) -> int:
    district_digits = 1e7
    county_digits = 1e12
    return int((cds_code % county_digits) / district_digits)

def get_schoolCode(cds_code: int) -> int:
    district_digits = 1e7
    return int(cds_code % district_digits)

def cds_code_split(cds_code: int) -> pd.Series:
    return pd.Series([get_countyCode(cds_code), get_districtCode(cds_code), get_schoolCode(cds_code)])

In [7]:
math24[["CountyCode", "DistrictCode", "SchoolCode"]] = math24.apply(lambda row: cds_code_split(int(row["cdsCode"])), axis=1)
math23[["CountyCode", "DistrictCode", "SchoolCode"]] = math23.apply(lambda row: cds_code_split(int(row["cdsCode"])), axis=1)
math22[["CountyCode", "DistrictCode", "SchoolCode"]] = math22.apply(lambda row: cds_code_split(int(row["cdsCode"])), axis=1)
math19[["CountyCode", "DistrictCode", "SchoolCode"]] = math19.apply(lambda row: cds_code_split(int(row["cdsCode"])), axis=1)
math18[["CountyCode", "DistrictCode", "SchoolCode"]] = math18.apply(lambda row: cds_code_split(int(row["cdsCode"])), axis=1)
math17[["CountyCode", "DistrictCode", "SchoolCode"]] = math17.apply(lambda row: cds_code_split(int(row["cdsCode"])), axis=1)

In [8]:
math = pd.concat([math24, math23, math22, math19, math18, math17]).drop_duplicates()

In [9]:
math

,cdsCode,currdenom,currstatus,studentgroup,Year,CountyCode,DistrictCode,SchoolCode
250,1316090131755,1.0,NaN,AA,2023-24,1,31609,131755
251,1316090131755,19.0,-168.1,ALL,2023-24,1,31609,131755
252,1316090131755,3.0,NaN,AS,2023-24,1,31609,131755
253,1316090131755,5.0,NaN,CAA,2023-24,1,31609,131755
254,1316090131755,8.0,NaN,EL,2023-24,1,31609,131755
...,...,...,...,...,...,...,...,...
148700,58727695838305,0.0,NaN,PI,2016-17,58,72769,5838305
148701,58727695838305,3.0,NaN,RFP,2016-17,58,72769,5838305
148702,58727695838305,54.0,-72.8,SED,2016-17,58,72769,5838305
148703,58727695838305,9.0,NaN,SWD,2016-17,58,72769,5838305


In [10]:
con = sqlite3.connect(DATABASE)
math.to_sql("Math", con, index=False)
con.close()